In [12]:
# # H1 – Gravidade (UTI + suporte) por estação do ano
# 
# Partimos de df_tipado.csv e respondemos:
# - Uso de UTI aumenta em outono/inverno?
# - Letalidade na ventilação invasiva é estável ou varia?
# - Há combinação de uso alto de UTI com letalidade baixa/alta?

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
# Aumenta o número máximo de colunas mostradas pelo pandas.
# Útil para inspecionar bases largas sem que ele corte com "..."
pd.set_option("display.max_columns", 120)

# Caminhos Utilizados
PROJECT_ROOT = Path.cwd().parent        # sobe um nível

DATA_RAW = PROJECT_ROOT / "srag_2023_analysis" / "data" / "processed" 
CSV_PATH = DATA_RAW / "df_tipado.csv"

In [3]:
df = pd.read_csv(CSV_PATH, low_memory=False)

In [4]:
# Garantir tipos
df["DT_SIN_PRI_date"] = pd.to_datetime(df["DT_SIN_PRI_date"], errors="coerce")
df["EVOLUCAO_BIN"] = pd.to_numeric(df["EVOLUCAO_BIN"], errors="coerce")
df["EVOLUCAO_disponivel"] = pd.to_numeric(df["EVOLUCAO_disponivel"], errors="coerce")


In [5]:
# === 2. Criar variável de estação (hemisfério sul) a partir da DT_SIN_PRI_date ===

def estacao_from_date(d):
    if pd.isna(d):
        return np.nan
    year = d.year
    outono = pd.Timestamp(year, 3, 20)
    inverno = pd.Timestamp(year, 6, 21)
    primavera = pd.Timestamp(year, 9, 23)
    verao2 = pd.Timestamp(year, 12, 22)
    if d < outono:
        return "verao"
    elif d < inverno:
        return "outono"
    elif d < primavera:
        return "inverno"
    elif d < verao2:
        return "primavera"
    else:
        return "verao"

df["estacao"] = df["DT_SIN_PRI_date"].map(estacao_from_date)

# Conferência rápida
print(df["estacao"].value_counts(dropna=False))

estacao
outono       97337
verao        64213
inverno      63509
primavera    54394
Name: count, dtype: int64


In [7]:
# === 3. Uso de UTI por estação ===
# UTI_flag: 1 = usou UTI, 0 = não, NaN = ignorado/missing

mask_uti_info = df["UTI_flag"].notna() & df["estacao"].notna()
df_uti = df.loc[mask_uti_info].copy()

uti_por_estacao = (
    df_uti
    .groupby("estacao")["UTI_flag"]
    .agg(taxa_uti="mean", n="count")
    .assign(taxa_uti_pct=lambda x: 100 * x["taxa_uti"])
    .sort_index()
)

print("\n=== Uso de UTI por estação (entre registros com UTI informada) ===")
print(uti_por_estacao)


=== Uso de UTI por estação (entre registros com UTI informada) ===
           taxa_uti      n  taxa_uti_pct
estacao                                 
inverno    0.299991  55925     29.999106
outono     0.294266  83975     29.426615
primavera  0.323072  47621     32.307175
verao      0.323700  56021     32.370004


In [8]:
# Também agrupar em "alta temporada" x "baixa"
alta = df_uti["estacao"].isin(["outono", "inverno"])
baixa = df_uti["estacao"].isin(["primavera", "verao"])

taxa_uti_alta = df_uti.loc[alta, "UTI_flag"].mean()
taxa_uti_baixa = df_uti.loc[baixa, "UTI_flag"].mean()

print("\nTaxa de UTI na alta temporada (outono+inverno): "
      f"{taxa_uti_alta*100:.1f}%")
print("Taxa de UTI na baixa temporada (primavera+verao): "
      f"{taxa_uti_baixa*100:.1f}%")


Taxa de UTI na alta temporada (outono+inverno): 29.7%
Taxa de UTI na baixa temporada (primavera+verao): 32.3%


In [9]:
# === 4. Letalidade geral por estação ===

mask_evol = df["EVOLUCAO_disponivel"] == 1
df_evol = df.loc[mask_evol].copy()

letal_por_estacao = (
    df_evol
    .groupby("estacao")["EVOLUCAO_BIN"]
    .agg(letalidade="mean", n="count")
    .assign(letalidade_pct=lambda x: 100 * x["letalidade"])
    .sort_index()
)

print("\n=== Letalidade geral por estação (EVOLUCAO_BIN) ===")
print(letal_por_estacao)



=== Letalidade geral por estação (EVOLUCAO_BIN) ===
           letalidade      n  letalidade_pct
estacao                                     
inverno      0.116181  59080       11.618145
outono       0.115138  90743       11.513836
primavera    0.146391  49846       14.639088
verao        0.144591  59727       14.459122


In [10]:
# === 5. Letalidade entre ventilação invasiva (SUPORT_VEN == 1) por estação ===

mask_inv = (df["EVOLUCAO_disponivel"] == 1) & (df["SUPORT_VEN"] == 1)
df_inv = df.loc[mask_inv].copy()

letal_inv_por_estacao = (
    df_inv
    .groupby("estacao")["EVOLUCAO_BIN"]
    .agg(letalidade="mean", n="count")
    .assign(letalidade_pct=lambda x: 100 * x["letalidade"])
    .sort_index()
)

print("\n=== Letalidade entre ventilação invasiva (SUPORT_VEN=1) por estação ===")
print(letal_inv_por_estacao)


=== Letalidade entre ventilação invasiva (SUPORT_VEN=1) por estação ===
           letalidade     n  letalidade_pct
estacao                                    
inverno      0.469244  5820       46.924399
outono       0.456260  9705       45.625966
primavera    0.521261  5456       52.126100
verao        0.508267  6290       50.826709


In [11]:
# === 6. Letalidade por UTI_flag dentro de cada estação (pra ver se há 'boa resposta' em algum período) ===

df_uti_evol = df.loc[mask_evol & df["UTI_flag"].notna()].copy()

letal_uti_estacao = (
    df_uti_evol
    .groupby(["estacao", "UTI_flag"])["EVOLUCAO_BIN"]
    .mean()
    .unstack("UTI_flag")
    .rename(columns={0.0: "letal_sem_uti", 1.0: "letal_com_uti"})
    .assign(
        letal_sem_uti_pct=lambda x: 100 * x["letal_sem_uti"],
        letal_com_uti_pct=lambda x: 100 * x["letal_com_uti"],
    )
    .sort_index()
)

print("\n=== Letalidade por UTI_flag dentro de cada estação ===")
print(letal_uti_estacao)


=== Letalidade por UTI_flag dentro de cada estação ===
UTI_flag   letal_sem_uti  letal_com_uti  letal_sem_uti_pct  letal_com_uti_pct
estacao                                                                      
inverno         0.068592       0.210060           6.859245          21.006011
outono          0.064180       0.232509           6.417980          23.250868
primavera       0.088454       0.252037           8.845413          25.203709
verao           0.089635       0.254248           8.963500          25.424844


1. O uso de UTI por caso de SRAG aumenta na alta temporada (outono/inverno)?
Não.

2. A letalidade entre os que usam ventilação invasiva é estável ou muda com a estação?

Entre os pacientes com SUPORT_VEN == 1 (ventilação invasiva) e desfecho conhecido:

Outono: letalidade 45,6%

Inverno: 46,9%

Primavera: 52,1%

Verão: 50,8%

Então:

A letalidade em ventilação invasiva é sempre muito alta (algo entre 45% e 52%).

Há uma variação moderada, com primavera/verão um pouco piores que outono/inverno (cerca de +5–6 pontos percentuais).

3. Há períodos em que o uso de UTI é alto mas a letalidade cai (boa resposta do sistema) ou UTI saturada com letalidade disparando?

Pelo que os dados mostram, não aparece um cenário de “UTI cheia e letalidade baixa”.

Estações com maior uso de UTI:

Primavera: ~32,1% dos casos em UTI, letalidade geral 14,6%

Verão: ~31,8% em UTI, letalidade geral 14,5%

Estações com uso um pouco menor de UTI:

Outono: ~29,1% em UTI, letalidade 11,5%

Inverno: ~29,9% em UTI, letalidade 11,6%

Ou seja:

Mais UTI → também mais letalidade (primavera/verão).

Menos UTI → letalidade menor (outono/inverno).

Mesmo dentro da UTI, a letalidade sobe um pouco em primavera/verão:

Letalidade em UTI (UTI_flag=1):

Outono/Inverno: ~21–23%

Primavera/Verão: ~25%

Então:

Não há evidência de “uso de UTI muito alto com letalidade caindo” (boa resposta com alta ocupação).

Também não dá pra provar “colapso de UTI” só com isso, mas o padrão é:

onde a fração de pacientes em UTI é maior, a letalidade também é maior,
sugerindo casos mais graves / perfil mais frágil nesses períodos (primavera/verão).

In [ ]:
# # H2 – Tempos de evolução (sintoma → internação/UTI/desfecho)
# 
# Calcula tempos em dias e compara:
# - distribuição global,
# - cura x óbito,
# - estação do ano.

In [14]:
df = pd.read_csv(CSV_PATH, low_memory=False)

In [15]:
# Converte datas principais
for col in ["DT_SIN_PRI_date", "DT_INTERNA_date", "DT_ENTUTI_date", "DT_EVOLUCA_date"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

df["EVOLUCAO_BIN"] = pd.to_numeric(df["EVOLUCAO_BIN"], errors="coerce")
df["EVOLUCAO_disponivel"] = pd.to_numeric(df["EVOLUCAO_disponivel"], errors="coerce")


In [16]:
# === 1. Diferenças em dias ===

df["dias_sin_interna"] = (df["DT_INTERNA_date"] - df["DT_SIN_PRI_date"]).dt.days
df["dias_sin_uti"]     = (df["DT_ENTUTI_date"] - df["DT_SIN_PRI_date"]).dt.days
df["dias_sin_evol"]    = (df["DT_EVOLUCA_date"] - df["DT_SIN_PRI_date"]).dt.days

# Limpeza simples de outliers absurdos
df["dias_sin_interna_cl"] = df["dias_sin_interna"].where(
    (df["dias_sin_interna"] >= 0) & (df["dias_sin_interna"] <= 60)
)
df["dias_sin_uti_cl"] = df["dias_sin_uti"].where(
    (df["dias_sin_uti"] >= 0) & (df["dias_sin_uti"] <= 60)
)
df["dias_sin_evol_cl"] = df["dias_sin_evol"].where(
    (df["dias_sin_evol"] >= 0) & (df["dias_sin_evol"] <= 120)
)

cols_tempo = ["dias_sin_interna_cl", "dias_sin_uti_cl", "dias_sin_evol_cl"]

print("\n=== Resumo robusto (global) dos tempos em dias ===")
print(df[cols_tempo].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]))



=== Resumo robusto (global) dos tempos em dias ===
       dias_sin_interna_cl  dias_sin_uti_cl  dias_sin_evol_cl
count        260082.000000     72455.000000     245293.000000
mean              3.491903         4.182003         12.405719
std               4.348482         5.159400         12.388582
min               0.000000         0.000000          0.000000
10%               0.000000         0.000000          3.000000
25%               1.000000         1.000000          5.000000
50%               2.000000         3.000000          9.000000
75%               5.000000         5.000000         15.000000
90%               7.000000         9.000000         25.000000
max              60.000000        60.000000        120.000000


In [17]:
# === 2. Tempos por desfecho (cura x óbito) ===

mask_evol = df["EVOLUCAO_disponivel"] == 1
tempo_por_desfecho = (
    df.loc[mask_evol]
    .groupby("EVOLUCAO_BIN")[cols_tempo]
    .median()
    .rename(index={0.0: "Cura", 1.0: "Óbito"})
)

print("\n=== Medianas dos tempos por desfecho (Cura x Óbito) ===")
print(tempo_por_desfecho)



=== Medianas dos tempos por desfecho (Cura x Óbito) ===
              dias_sin_interna_cl  dias_sin_uti_cl  dias_sin_evol_cl
EVOLUCAO_BIN                                                        
Cura                          3.0              3.0               9.0
Óbito                         2.0              3.0              10.0


In [18]:
# === 3. Estação do ano ===

def estacao_from_date(d):
    if pd.isna(d):
        return np.nan
    year = d.year
    outono = pd.Timestamp(year, 3, 20)
    inverno = pd.Timestamp(year, 6, 21)
    primavera = pd.Timestamp(year, 9, 23)
    verao2 = pd.Timestamp(year, 12, 22)
    if d < outono:
        return "verao"
    elif d < inverno:
        return "outono"
    elif d < primavera:
        return "inverno"
    elif d < verao2:
        return "primavera"
    else:
        return "verao"

df["estacao"] = df["DT_SIN_PRI_date"].map(estacao_from_date)

tempo_por_estacao = (
    df.loc[mask_evol]
    .groupby("estacao")[cols_tempo]
    .median()
    .sort_index()
)

print("\n=== Medianas dos tempos por estação ===")
print(tempo_por_estacao)


=== Medianas dos tempos por estação ===
           dias_sin_interna_cl  dias_sin_uti_cl  dias_sin_evol_cl
estacao                                                          
inverno                    3.0              3.0               9.0
outono                     3.0              3.0               9.0
primavera                  2.0              3.0               9.0
verao                      2.0              3.0               8.0
